# 검수하는 자동화 코드 작성

In [2]:
import pandas as pd
import re
import json
import random
import csv
from tqdm.notebook import tqdm
from collections import Counter

In [18]:
df = pd.read_csv("Curriculum_Mid.csv")
df_elem = pd.read_csv("Curriculum_Elem.csv")
df_high =pd.read_csv("Curriculum_High.csv")

In [4]:
df.columns

Index(['textbook ID', '학년', '출판사', '교육과정', 'Lesson', '구분', 'Title',
       'DESCRIPTION', 'Key Expression (영어)', 'Key Expression (한국어)',
       'Sub Expression (영어)', 'Sub Expression (한국어)', '학습목표', 'Quiz 1',
       'Korean 1', 'Option 1-1', 'Option 1-2', 'Option 1-3', 'Answer 1',
       'Index 1', 'Quiz 2', 'Korean 2', 'Option 2-1', 'Option 2-2',
       'Option 2-3', 'Answer 2', 'Index 2', '샘플 스크립트 (영어)', '샘플 스크립트 (한국어)'],
      dtype='object')

In [ ]:
# # csv 파일을 json 파일로 변환
sample_df_json = df.to_json(orient="records", force_ascii=False, indent=2)

sample_df_json = json.loads(sample_df_json)
print(type(sample_df_json))
sample_df_json[5]

# csv 파일을 json 파일로 변환
df_json = df.to_json(orient="records", force_ascii=False, indent=2)

df_json = json.loads(df_json)
print(type(df_json))
df_json[5]

<class 'list'>
<class 'list'>


{'textbook ID': 701,
 '학년': '중1',
 '출판사': '능률(김)',
 '교육과정': 2015,
 'Lesson': 3,
 '구분': 'function',
 'Title': 'Spend Smart, Save Smart',
 'DESCRIPTION': 'Can I ~?를 활용해 허락을 요청하고 답하기.',
 'Key Expression (영어)': 'A: Can I borrow your book?\n B: Yes, of course. / I’m sorry, but you can’t.',
 'Key Expression (한국어)': 'A: 네 책을 빌릴 수 있을까?\n B: 네, 물론이야. / 미안하지만 안 돼.',
 'Sub Expression (영어)': 'A: May I ~?\n B: Sure, go ahead. / Sorry, you can’t.\n A: Is it okay if I ~?\n B: Of course. / I’m afraid not.',
 'Sub Expression (한국어)': 'A: ~ 해도 될까? \n B: 그래, 해봐. / 미안, 안 돼. \n A: ~ 해도 괜찮을까? \n B: 당연하지. / 미안, 안 돼.',
 '학습목표': '허락을 요청하고 응답하는 표현을 사용할 수 있어요.\n Can I ~?와 같은 표현을 활용해 허락을 요청하고 답할 수 있어요.',
 'Quiz 1': '___ I borrow your book?',
 'Korean 1': '내 책 좀 빌려도 될까요?',
 'Option 1-1': 'May',
 'Option 1-2': 'Can',
 'Option 1-3': 'Might',
 'Answer 1': 'Can',
 'Index 1': 2,
 'Quiz 2': 'May I ___ your eraser?',
 'Korean 2': '내 지우개를 빌릴 수 있을까요?',
 'Option 2-1': 'buy',
 'Option 2-2': 'borrow',
 'Option 2-3': 'lend',
 '

In [ ]:
class ValidClass:
    def __init__(self, data:pd.DataFrame) -> pd.DataFrame:
        # CSV 파일 json로 변환
        csv_to_json = data.to_json(orient="records", force_ascii=False, indent=2)

        # json 파일로 불러오기
        self.data_json = json.loads(csv_to_json)

        # index 초기화 변수
        self.index_1_num = 0
        self.index_2_num = 0
        self.index_3_num = 0

    def check_valid_quiz(self, quiz:dict) -> bool:
        # 에러 이유 저장할 리스트
        reason_error_quiz1 = []
        reason_error_quiz2 = []

        # 변수 가져오기 quiz1
        quiz_1 = quiz.get("Quiz 1")
        korean_1 = quiz.get("Korean 1")
        option_1_1 = quiz.get("Option 1-1")
        option_1_2 = quiz.get("Option 1-2")
        option_1_3 = quiz.get("Option 1-3")
        answer_1 = quiz.get("Answer 1")
        index_1 = quiz.get("Index 1")
        
        # 변수 가져오기 quiz2
        quiz_2 = quiz.get("Quiz 2")
        korean_2 = quiz.get("Korean 2")
        option_2_1 = quiz.get("Option 2-1")
        option_2_2 = quiz.get("Option 2-2")
        option_2_3 = quiz.get("Option 2-3")
        answer_2 = quiz.get("Answer 2")
        index_2 = quiz.get("Index 2")

        # 0. 필드값 누락 체크
        if (
            not quiz_1
            or not korean_1
            or not option_1_1
            or not option_1_2
            or not option_1_3
            or not answer_1
            or not index_1
            or not quiz_2
            or not korean_2
            or not option_2_1
            or not option_2_2
            or not option_2_3
            or not answer_2
            or not index_2
        ):
            return False

        # QUIZ 1
        # 1. 빈칸이 하나 이상있고, 두 개 이상이 없는지 체크
        appropriate_blank = r"_+"
        underscores = re.findall(appropriate_blank, quiz_1)

        if len(underscores) > 1:
            reason_error_quiz1.append("Blank error: 밑줄이 두 개 이상 존재")
        elif len(underscores) == 1:
            pass
        else:
            reason_error_quiz1.append("Blank error: 밑줄이 존재하지 않음")
        
        # 2. 정답이 옵션에 포함되어 있는지 체크
        options = [option_1_1, option_1_2, option_1_3]
        
        # answer가 options에 있는지 확인
        if answer_1 not in options:
            reason_error_quiz1.append("Answer in options error : answer 확인")
        
        # answer가 숫자로 이루어져있는지 확인
        if isinstance(index_1, str) and not index_1.isnumeric():
            reason_error_quiz1.append("Answer in options error : answer type 확인")

        # 3. 정답의 인덱스가 answer_index와 일치하는지 확인
        index_1 = int(index_1)
        if options[index_1 - 1] != answer_1:
            reason_error_quiz1.append("Answer index not match option index : index 확인")

        # 4. answer_index의 분포 검수
        if index_1 == 1:
            self.index_1_num += 1
        elif index_1 == 2:
            self.index_2_num += 1
        elif index_1 == 3:
            self.index_3_num += 1

        # 에러가 발생하면 데이터에 reason_error 추가
        if reason_error_quiz1:
            quiz["reason_error_quiz1"] = reason_error_quiz1  # 에러 이유 추가
            return False
        
        # QUIZ 2
        # 1. 빈칸이 하나 이상있고, 두 개 이상이 없는지 체크
        appropriate_blank = r"_+"
        underscores = re.findall(appropriate_blank, quiz_2)

        if len(underscores) > 1:
            reason_error_quiz2.append("Blank error: 밑줄이 두 개 이상 존재")
        elif len(underscores) == 1:
            pass
        else:
            reason_error_quiz2.append("Blank error: 밑줄이 존재하지 않음")
        
        # 2. 정답이 옵션에 포함되어 있는지 체크
        options = [option_2_1, option_2_2, option_2_3]
        
        # answer가 options에 있는지 확인
        if answer_2 not in options:
            reason_error_quiz2.append("Answer in options error : answer 확인")
        
        # answer가 숫자로 이루어져있는지 확인
        if isinstance(index_2, str) and not index_2.isnumeric():
            reason_error_quiz2.append("Answer in options error : answer type 확인")

        # 3. 정답의 인덱스가 answer_index와 일치하는지 확인
        index_2 = int(index_2)
        if options[index_2 - 1] != answer_2:
            reason_error_quiz2.append("Answer index not match option index : index 확인")

        # 4. answer_index의 분포 검수
        if index_2 == 1:
            self.index_1_num += 1
        elif index_2 == 2:
            self.index_2_num += 1
        elif index_2 == 3:
            self.index_3_num += 1

        # 에러가 발생하면 데이터에 reason_error 추가
        if reason_error_quiz1:
            quiz["reason_error_quiz2"] = reason_error_quiz2  # 에러 이유 추가
            return False
        
    def check_index_distribution(self):
        total_index_num = self.index_1_num + self.index_2_num + self.index_3_num

        if total_index_num > 0:
            index_percent_1 = (self.index_1_num / total_index_num) * 100
            index_percent_2 = (self.index_2_num / total_index_num) * 100
            index_percent_3 = (self.index_3_num / total_index_num) * 100
        
        if not (25 <= index_percent_1 <= 35 and 25 <= index_percent_2 <= 35 and 25 <= index_percent_3 <= 35):   
            print(f"Distribution Error: {total_index_num} (1: {self.index_1_num}, 2: {self.index_2_num}, 3: {self.index_3_num})")
    
    def normalization(self, input_file_name:str, output_file_name:str) -> dict:
        # 데이터 가져오기
        quiz = pd.read_csv(input_file_name)

        # 변수 random하게 만들기
        quiz_1_random_index = [random.randint(1,3) for x in range(len(quiz))]
        quiz_2_random_index = [random.randint(1,3) for x in range(len(quiz))]

        # 파일명 불러오기
        column_names = quiz.columns.tolist()

        with open(output_file_name, "w", encoding="utf-8-sig", newline="") as fw:
            writer = csv.DictWriter(fw, fieldnames = column_names)
            writer.writeheader()
            for i in tqdm(range(len(quiz))):
                row = quiz.iloc[i]
                target_quiz_1_index = quiz_1_random_index[i]
                target_quiz_2_index = quiz_2_random_index[i]

                quiz_1_options = [row['Option 1-1'], row['Option 1-2'], row['Option 1-3']]
                quiz_1_answer_index = int(row['Index 1'])
                quiz_1_answer = row["Answer 1"]


                quiz_2_options = [row['Option 2-1'], row['Option 2-2'], row['Option 2-3']]
                quiz_2_answer_index = int(row['Index 2'])
                quiz_2_answer = row["Answer 2"]

                if quiz_1_answer_index != target_quiz_1_index:
                    quiz_1_tmp_options = quiz_1_options[target_quiz_1_index - 1] # 원본 index 보존
                    quiz_1_options[target_quiz_1_index - 1] = quiz_1_answer
                    quiz_1_options[quiz_1_answer_index - 1] = quiz_1_tmp_options

                if quiz_2_answer_index != target_quiz_2_index:
                    quiz_2_tmp_options = quiz_2_options[target_quiz_2_index - 1] # 원본 index 보존
                    quiz_2_options[target_quiz_2_index - 1] = quiz_2_answer
                    quiz_2_options[quiz_2_answer_index - 1] = quiz_2_tmp_options  

                row_dict = row.to_dict()
                # quiz 1 덮어씌우기
                row_dict["Option 1-1"] = quiz_1_options[0]
                row_dict["Option 1-2"] = quiz_1_options[1]
                row_dict["Option 1-3"] = quiz_1_options[2]
                row_dict["Index 1"] = target_quiz_1_index
                row_dict["Option 2-1"] = quiz_2_options[0]
                row_dict["Option 2-2"] = quiz_2_options[1]
                row_dict["Option 2-3"] = quiz_2_options[2]
                row_dict["Index 2"] = target_quiz_2_index

                writer.writerow(row_dict)

        return print(f"csv 파일 완료")
    
    def save_to_json(self, file_name):
        # 수정된 퀴즈 데이터를 JSON 파일로 다시 저장
        with open(file_name, 'w', encoding="utf-8-sig") as f:
            json.dumps(self.data_json, f, ensure_ascii=False, indent=2)
        print(f"JSON 파일 저장 완료 : {file_name}")
        
            




In [39]:
valid = Valid(df_high)

valid.normalization(input_file_name="Curriculum_High.csv", output_file_name="ReIndex_High_Curriculum.csv")

  0%|          | 0/679 [00:00<?, ?it/s]

csv 파일 완료


In [30]:
asdf = pd.read_csv("ReIndex_Elem_Curriculum.csv")
print(asdf['Index 1'].value_counts())
print("\n")
print(asdf['Index 2'].value_counts())

Index 1
1    245
3    218
2    207
Name: count, dtype: int64


Index 2
3    231
2    231
1    208
Name: count, dtype: int64


In [ ]:
va

TypeError: Valid.check_index_distribution() takes 1 positional argument but 2 were given

In [ ]:
# valid = Valid(sample_df)

# for i in range(len(sample_df)):
#     b = valid.data_json[i]
#     c = valid.check_valid_quiz(b)
#     d = valid.normalization(b)
#     print(b)
#     print(d)

In [ ]:
# valid = Valid(sample_df)
# normalized_quizzes = [valid.normalization(quiz) for quiz in valid.data_json]
# print(normalized_quizzes)
# # valid.save_to_json("normalized_quiz.json")

In [ ]:
# normalized_quizzes = [quiz for quiz in (valid.normalization(quiz) for quiz in valid.data_json) if quiz]
# valid.data_json = normalized_quizzes
# valid.data_json

# Fill in the blank - LLM

In [ ]:
from pydantic import BaseModel
from openai import OpenAI
from openai.lib._parsing import type_to_response_format_param
import json
from getpass import getpass
from jinja2 import Template

openai_api_key = getpass("OPENAI_API_KEY")
client = OpenAI(api_key=openai_api_key)

In [ ]:
prompt_template = Template("""
[PURPOSE]
You are going to solve the quiz and write down the answer_reason.
There is not necessarily a single correct answer.
Select all that are correct. 
                           
[Example]
```
Input: 
    {
    "quiz": "What is your ______ sport?",
    "korean": "네가 가장 좋아하는 스포츠는 무엇이에요?",
    "option_1": "like",
    "option_2": "favorite",
    "option_3": "best",
  }

Output:
    {   
    "possible_answer": "favorite",
    "answer_reason": "The word 'favorite' is the correct answer because the sentence is asking about someone's most preferred sport.

1. 'favorite' means 가장 좋아하는 in Korean, which matches the meaning of the given Korean sentence.
2. 'like' is a verb (좋아하다) and does not fit grammatically in this sentence.
3. 'best' means 최고의, which refers to the highest quality rather than personal preference.

Thus, 'favorite' is the correct choice."
}
```                     
```
Input: 
    {
    "quiz": "A: What are you _____ on your holiday? B: I’m going to travel.",
    "korean": "A: 너는 휴일에 무엇을 할 예정이니? B: 여행을 갈 거야.",
    "option_1": "planning to do",
    "option_2": "going to do",
    "option_3": "will do",
  }

Output:
    {   
    "possible_answer": "planning to do, going to do",
    "answer_reason": "Both 'planning to do' and 'going to do' fit naturally in the sentence.

1. 'Going to do' is commonly used for future plans and matches the response "I’m going to travel."
2. 'Planning to do' also makes sense because it refers to making arrangements for the holiday.
3. 'Will do' is less natural in this context because it’s not commonly used for asking about planned activities."

Thus, 'planning to do' and 'going to do' are the correct choice."
}
```

[Input]
```
QUIZ : {{quiz}}
KOREAN : {{korean}}
OPTION_1 : {{option_1}}
OPTION_2 : {{option_2}}
OPTION_3 : {{option_3}}
```
""")

In [ ]:
prompt = prompt_template.render(
    quiz = "___ I borrow your book?",
    korean = "내 책 좀 빌려도 될까요?",
    option_1 = "May",
    option_2 = "Can",
    option_3 = "Might"
)

In [ ]:
print(prompt)

In [ ]:
class Reason(BaseModel):
    possible_answer  : list[str]
    answer_reason : str

In [ ]:
response_format = type_to_response_format_param(Reason)

In [ ]:
response_format

In [ ]:
def completion(prompt : str) -> str:
    response = client.beta.chat.completions.parse(
        model = 'o3-mini',
        reasoning_effort='low',
        messages = [
            {"role" : "system", "content" : "You are going to solve the quiz and write down the answer_reason."},
            {"role" : "user", "content" : prompt}
        ],
        response_format = Reason,
    )
    return response.choices[0].message.parsed

In [ ]:
response = completion(prompt)

In [ ]:
response

In [ ]:
response_output = json.dumps(response.dict(), ensure_ascii=False, indent=2)
print(response_output)

In [ ]:
type(response_output)

In [ ]:
print('f')